# CS534 Homework 3

Put your homework in the directory with your name. Please mentionin this file the names of any students with whom you collaborated. If you didn't collaborate with anyone, mark your collaborators as "None." Remember, your goal is to communicate. Full credit will be given only to correct solutions which are described clearly. Convoluted and obtuse descriptions will receive low marks. To complete your homework, you may ONLY consult the following material:

lecture slides course notes you or others took during lecture. the required text (CLRS) websites that may clarify the concepts covered in the material but do not in any way provide complete solutions to the problems. Deadline 2/20/2019

Please provide an answer to the following question:

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [279]:
data = pd.read_csv("iris.csv",header=None)
data.columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
data_val=data.values

def classify(s):
    if s=='Iris-versicolor':
        return 1
    else:
        return 0

X=data_val[:,0:3]
y=data_val[:,4]

y1=np.array([classify(x) for x in y])


# Question 1 (15 pts)

To classify iris dataset (Iris-Versicolor vs. others) in the best way, you have to create an algorithm able to determine (with the k-fold cross validation) what is the best space transformation (among rbf_kernel, polynomial features, and polynomial kernel) and its hyperparameters. Each transformation has its own parameters rbf_kernel->gamma, polynomial-features-> degree, polynomial_kernel ->(gamma, degree). The performance has to be tested of the entire algorithm with another K-fold cross-validation (please use then trainset, validationset, and testset).

In [315]:
from sklearn import svm
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support,accuracy_score
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import polynomial_kernel
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import f1_score


def transform_rbf(hyper1, X_train, Y_train, skf, n_splits):
    rbf_dict={}
    for gamma in hyper1:
        #print("gamma=",gamma)
        #n_splits=10
        #skf = StratifiedKFold(n_splits=n_splits,random_state=10,shuffle=True)
        #skf = KFold(n_splits=n_splits,random_state=10,shuffle=True)
        c=0
        for train_index, val_index in skf.split(X_train, Y_train):
            X_train1=X_train[train_index]
            X_val=X_train[val_index]
            y_train=Y_train[train_index]
            y_val=Y_train[val_index]
            scaler = StandardScaler()
            XS_train=scaler.fit_transform(X_train1) 
            XS_val=scaler.transform(X_val)
            XSK_train=rbf_kernel(XS_train, gamma=gamma)
            XSK_test=rbf_kernel(XS_val,XS_train, gamma=gamma)
            
            clf = LogisticRegression(random_state=0, solver='lbfgs').fit(XSK_train,y_train)
            y_pred=clf.predict(XSK_test) 
            #print(gamma,accuracy_score(y_val, y_pred))
            c+=accuracy_score(y_val, y_pred)
            #c+=f1_score(y_val, y_pred, average="macro")
            
        accuracyavg=c/n_splits
        rbf_dict[gamma]=accuracyavg
   
    # Cycle through dictionary to highlight hyperparameter with highest metric (acc, F1 score, whatever)    
    max=0
    for key,value in rbf_dict.items():
        #print(key,value)
        if value > max:
            max=value
            max_key=key
    #best_tup=(max_key,max)
    
    return max_key


def val_rbf(X_train, Y_train,skf,gamma,n_splits):
    
    c=0
    for train_index, val_index in skf.split(X_train, Y_train):
        X_train1=X_train[train_index]
        X_val=X_train[val_index]
        y_train=Y_train[train_index]
        y_val=Y_train[val_index]
        scaler = StandardScaler()
        XS_train=scaler.fit_transform(X_train1) 
        XS_val=scaler.transform(X_val)
        XSK_train=rbf_kernel(XS_train, gamma=gamma)
        XSK_test=rbf_kernel(XS_val,XS_train, gamma=gamma)

        clf = LogisticRegression(random_state=0, solver='lbfgs').fit(XSK_train,y_train)
        y_pred=clf.predict(XSK_test) 
        #print(gamma,accuracy_score(y_val, y_pred))
        c+=accuracy_score(y_val, y_pred)
        #c+=f1_score(y_val, y_pred, average="macro")

    accuracyavg=c/n_splits
    return accuracyavg

def classifier_rbf(gamma_val,X_train,X_test,y_train,y_test):
    scaler = StandardScaler()
    XS_train=scaler.fit_transform(X_train) 
    XS_test=scaler.transform(X_test)
    XSK_train=rbf_kernel(XS_train, gamma=gamma_val)
    XSK_test=rbf_kernel(XS_test,XS_train, gamma=gamma_val)

    clf = LogisticRegression(random_state=0, solver='lbfgs').fit(XSK_train,y_train)
    y_pred=clf.predict(XSK_test) 
    hyper_param=gamma_val
    string="rbf_kernel"
    return accuracy_score(y_test, y_pred),string
    #return f1_score(y_test, y_pred,average="macro"),string
    
            
def transform_poly(hyper1, hyper2, X_train, Y_train, skf, n_splits):
    # hyper1=degree, hyper2=gamma
    poly_dict={}
    param_list = [(degree,gamma) for degree in hyper1 for gamma in hyper2]
    param_tup=tuple(param_list)
    for tup in param_tup:
        #n_splits=10
        #skf = StratifiedKFold(n_splits=n_splits,random_state=10,shuffle=True)
        #skf = KFold(n_splits=n_splits,random_state=10,shuffle=True)
        c=0
        for train_index, val_index in skf.split(X_train, Y_train):
            X_train1=X_train[train_index]
            X_val=X_train[val_index]
            y_train=Y_train[train_index]
            y_val=Y_train[val_index]
            scaler = StandardScaler()
            XS_train=scaler.fit_transform(X_train1) 
            XS_val=scaler.transform(X_val)
            degree,gamma=tup
            XSK_train=polynomial_kernel(XS_train,degree=degree, gamma=gamma)
            XSK_test=polynomial_kernel(XS_val,XS_train,degree=degree, gamma=gamma)
            
            clf = LogisticRegression(random_state=0, solver='lbfgs').fit(XSK_train,y_train)
            y_pred=clf.predict(XSK_test) 
            #print(gamma,accuracy_score(y_val, y_pred))
            c+=accuracy_score(y_val, y_pred)
            #c+=f1_score(y_val, y_pred, average="macro")
            
        accuracyavg=c/n_splits
        poly_dict[tup]=accuracyavg
        
    # Cycle through dictionary to highlight hyperparameter with highest metric (acc, F1 score, whatever)    
    max=0
    for key,value in poly_dict.items():
        #print(key,value)
        if value > max:
            max=value
            max_key=key
    #best_tup=(max_key,max)
    
    return max_key

def val_poly( X_train, Y_train,skf, param_tup, n_splits):
    # hyper1=degree, hyper2=gamma
    degree,gamma=param_tup
    c=0
    for train_index, val_index in skf.split(X_train, Y_train):
        X_train1=X_train[train_index]
        X_val=X_train[val_index]
        y_train=Y_train[train_index]
        y_val=Y_train[val_index]
        scaler = StandardScaler()
        XS_train=scaler.fit_transform(X_train1) 
        XS_val=scaler.transform(X_val)
        degree,gamma=tup
        XSK_train=polynomial_kernel(XS_train,degree=degree, gamma=gamma)
        XSK_test=polynomial_kernel(XS_val,XS_train,degree=degree, gamma=gamma)

        clf = LogisticRegression(random_state=0, solver='lbfgs').fit(XSK_train,y_train)
        y_pred=clf.predict(XSK_test) 
        #print(gamma,accuracy_score(y_val, y_pred))
        c+=accuracy_score(y_val, y_pred)
        #c+=f1_score(y_val, y_pred, average="macro")

    accuracyavg=c/n_splits
    return accuracyavg

def classifier_polynomial_kernel(param_tup,X_train,X_test,y_train,y_test):
    degree_val,gamma_val=param_tup
    scaler = StandardScaler()
    XS_train=scaler.fit_transform(X_train) 
    XS_test=scaler.transform(X_test)
    XSK_train=polynomial_kernel(XS_train, degree=degree_val,gamma=gamma_val)
    XSK_test=polynomial_kernel(XS_test,XS_train,degree=degree_val,gamma=gamma_val)

    clf = LogisticRegression(random_state=0, solver='lbfgs').fit(XSK_train,y_train)
    y_pred=clf.predict(XSK_test) 
    string="Poly_kernel"
    return accuracy_score(y_test, y_pred),string
    #return f1_score(y_test, y_pred,average="macro"),string

def transform_poly_features(hyper1, X_train, Y_train,skf,n_splits):
    poly_feat_dict={}
    for degree in hyper1:
        #print("gamma=",gamma)
        #n_splits=10
        #skf = StratifiedKFold(n_splits=n_splits,random_state=10,shuffle=True)
        #skf = KFold(n_splits=n_splits,random_state=10,shuffle=True)
        c=0
        for train_index, val_index in skf.split(X_train, Y_train):
            X_train1=X_train[train_index]
            X_val=X_train[val_index]
            y_train=Y_train[train_index]
            y_val=Y_train[val_index]
            scaler = StandardScaler()
            XS_train=scaler.fit_transform(X_train1) 
            XS_val=scaler.transform(X_val)
            poly = PolynomialFeatures(degree)
            XSK_train=poly.fit_transform(XS_train)
            XSK_test=poly.transform(XS_val)
            
            clf = LogisticRegression(random_state=0, solver='lbfgs').fit(XSK_train,y_train)
            y_pred=clf.predict(XSK_test) 
            #print(gamma,accuracy_score(y_val, y_pred))
            c+=accuracy_score(y_val, y_pred)
            #c+=f1_score(y_val, y_pred, average="macro")
            
        accuracyavg=c/n_splits
        poly_feat_dict[degree]=accuracyavg
   
    # Cycle through dictionary to highlight hyperparameter with highest metric (acc, F1 score, whatever)    
    max=0
    for key,value in poly_feat_dict.items():
        #print(key,value)
        if value > max:
            max=value
            max_key=key
    #best_tup=(max_key,max)
    
    return max_key

def val_poly_features(X_train, Y_train,skf, degree, n_splits):
    
    c=0
    for train_index, val_index in skf.split(X_train, Y_train):
        X_train1=X_train[train_index]
        X_val=X_train[val_index]
        y_train=Y_train[train_index]
        y_val=Y_train[val_index]
        scaler = StandardScaler()
        XS_train=scaler.fit_transform(X_train1) 
        XS_val=scaler.transform(X_val)
        poly = PolynomialFeatures(degree)
        XSK_train=poly.fit_transform(XS_train)
        XSK_test=poly.transform(XS_val)

        clf = LogisticRegression(random_state=0, solver='lbfgs').fit(XSK_train,y_train)
        y_pred=clf.predict(XSK_test) 
        #print(gamma,accuracy_score(y_val, y_pred))
        c+=accuracy_score(y_val, y_pred)
        #c+=f1_score(y_val, y_pred, average="macro")

    accuracyavg=c/n_splits
    return accuracyavg

def classifier_polynomial_features(degree,X_train,X_test,y_train,y_test):
    scaler = StandardScaler()
    XS_train=scaler.fit_transform(X_train) 
    XS_test=scaler.transform(X_test)
    poly = PolynomialFeatures(degree)
    XSK_train=poly.fit_transform(XS_train)
    XSK_test=poly.transform(XS_test)

    clf = LogisticRegression(random_state=0, solver='lbfgs').fit(XSK_train,y_train)
    y_pred=clf.predict(XSK_test) 
    string="Poly_Features"
    
    return accuracy_score(y_test, y_pred),string
    #return f1_score(y_test, y_pred,average="macro"),string

In [316]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.2)

gamma = [2,3,4,5,6,7,8,9,10,15,20,25,30,35,40,45,50]
gamma_rbf = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3.0,4,5,6,7,8,9,10]
degree= [2,3,4,5,6,7,8,9,10,15,20,25]
#degree= [2,3]
#print(type(gamma))
#gamma_val=kernel_rbf(gamma_rbf,X_train,y_train)

log_dict={}
val_log_dict={}

n_splits=10
skf = KFold(n_splits=n_splits,shuffle=True)

#Hyperparameter tuning.
hyper_param_tup_rbf=transform_rbf(gamma_rbf,X_train,y_train,skf,n_splits)
hyper_param_tup_polyk=transform_poly(degree,gamma_rbf,X_train, y_train,skf,n_splits)
hyper_param_tup_poly_feat=transform_poly_features(degree, X_train, y_train,skf,n_splits)


skf1 = KFold(n_splits=n_splits,shuffle=True)

#Validation using K fold CV.
rbf_val_acc=val_rbf(X_train, y_train,skf1,hyper_param_tup_rbf,n_splits)
print("Validation Accuracy of rbf_kernel =", rbf_val_acc)
polyk_val_acc=val_poly( X_train, y_train,skf1, hyper_param_tup_polyk, n_splits)
print("Validation Accuracy of polynomial_kernel =", polyk_val_acc)
poly_feat_val_acc=val_poly_features(X_train, y_train,skf1, hyper_param_tup_poly_feat, n_splits)
print("Validation Accuracy of polynomial_features =", poly_feat_val_acc)

print() # for formatting
## Running the algo on the test data
rbf_acc,string = classifier_rbf(hyper_param_tup_rbf,X_train,X_test,y_train,y_test)
print("Accuracy of rbf_kernel=%s,gamma=%s " %(rbf_acc,hyper_param_tup_rbf))
key1=(string,hyper_param_tup_rbf)
log_dict[key1]=rbf_acc
val_log_dict[key1]=rbf_val_acc

polyk_acc,string=classifier_polynomial_kernel(hyper_param_tup_polyk,X_train,X_test,y_train,y_test)
print("Accuracy of polynomial_kernel =%s,degree=%s, gamma=%s " %(polyk_acc,hyper_param_tup_polyk[0], hyper_param_tup_polyk[1]))
key2=(string,hyper_param_tup_polyk)
log_dict[key2]=polyk_acc
val_log_dict[key2]=polyk_val_acc

poly_feat_acc,string=classifier_polynomial_features(hyper_param_tup_poly_feat,X_train,X_test,y_train,y_test)
print("Accuracy of polynomial_features =%s,degree=%s" %(poly_feat_acc,hyper_param_tup_poly_feat))
key3=(string,hyper_param_tup_poly_feat)
log_dict[key3]=poly_feat_acc
val_log_dict[key3]=poly_feat_val_acc




C:\Users\sayyapureddi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Validation Accuracy of rbf_kernel = 0.883333333333
Validation Accuracy of polynomial_kernel = 0.95
Validation Accuracy of polynomial_features = 0.916666666667

Accuracy of rbf_kernel=0.833333333333,gamma=0.8 
Accuracy of polynomial_kernel =0.966666666667,degree=2, gamma=2.3 
Accuracy of polynomial_features =0.933333333333,degree=2


In [317]:
print(log_dict)

{('rbf_kernel', 0.8): 0.83333333333333337, ('Poly_kernel', (2, 2.3)): 0.96666666666666667, ('Poly_Features', 2): 0.93333333333333335}


## The polynomial_kernel seems to have the highest accuracy.

## Question 2 (10 pts)
Write the explicit constraints (without using any vectorial notation, as a summation of single variables multiplied by a constant + bias term) of the Support Vector Machine to classify correctly iris dataset (Iris-Versicolor vs. others). In particular use 5 points in Iris-Versicolor, 2 points for iris-setosa, and 3 points for iris Virginia.
Please show the points you selected and after the constraints

In [11]:
data = pd.read_csv("iris.csv",header=None)
data.columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [24]:
data_versicolor =data[data["class"]=='Iris-versicolor'].sample(5)
data_setosa =data[data["class"]=='Iris-setosa'].sample(2)
data_virginica =data[data["class"]=='Iris-virginica'].sample(3)
data_sample=pd.concat([data_versicolor,data_setosa,data_virginica])

### Selected points are as follows

In [23]:
data_sample

,sepal_length,sepal_width,petal_length,petal_width,class
66,5.6,3.0,4.5,1.5,Iris-versicolor
62,6.0,2.2,4.0,1.0,Iris-versicolor
70,5.9,3.2,4.8,1.8,Iris-versicolor
69,5.6,2.5,3.9,1.1,Iris-versicolor
82,5.8,2.7,3.9,1.2,Iris-versicolor
11,4.8,3.4,1.6,0.2,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
115,6.4,3.2,5.3,2.3,Iris-virginica
106,4.9,2.5,4.5,1.7,Iris-virginica
118,7.7,2.6,6.9,2.3,Iris-virginica


### The below equations show how the weights and the bias are calculated from the selected points.

### Versicolor =1

$$\ w1*5.6 + w2*3.0 + w3*4.5 + w4*1.5 + b \geq  1 $$
$$\ w1*6.0 + w2*2.2 + w3*4.0 + w4*1.0 + b \geq  1 $$
$$\ w1*5.9 + w2*3.2 + w3*4.8 + w4*1.8 + b \geq  1 $$
$$\ w1*5.6 + w2*2.5 + w3*3.9 + w4*1.1 + b \geq  1 $$
$$\ w1*5.8 + w2*2.7 + w3*3.9 + w4*1.1 + b \geq  1 $$

### Others =-1
$$\ w1*4.8 + w2*3.4 + w3*1.6 + w4*0.2 + b \leq -1 $$
$$\ w1*4.6 + w2*3.4 + w3*1.4 + w4*0.3 + b \leq -1 $$
$$\ w1*6.4 + w2*3.2 + w3*5.3 + w4*2.3 + b \leq -1 $$
$$\ w1*4.9 + w2*2.5 + w3*4.5 + w4*1.7 + b \leq -1 $$
$$\ w1*7.7 + w2*2.6 + w3*6.9 + w4*2.3 + b \leq -1 $$

# Extra Points ( 5pts)
An unbalanced dataset (e.g. 95% vs 5%) can be problematic even in the training phase. The learned function can be trivial, e.g. always predicting one class.
A possible solution can have a weight for each point in the way that making a mistake in the minority class will coun more w.r.t. the other. Please redefine the likelihood of the logistic regression to consider these weights for each point. Please compute the log-likelihood and its derivatives.
In addition, add to the negative log-likelihood the norm of W (sum of the square of each component) and compute the derivatives. 

### The Anwser to this question is attached as a seperate scanned file in the submission folder